In [ ]:
# Instalamos Selenium y el driver de Chrome para realizar pruebas con el respectivo navegador

!pip install selenium
!apt get update 
!apt install chronium-chromedriver
!cp /usr/lib/chronium-browser/chromedriver /usr/bin

E: Invalid operation get
Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package chronium-chromedriver
cp: cannot stat '/usr/lib/chronium-browser/chromedriver': No such file or directory


In [ ]:
# Instalamos Kora para importar funciones de un .ipynb a otro .ipynb en Google COLAB
!pip install kora -q

In [ ]:
# Una vez instaladas las librerias necesarias, procedemos a su llamado para este proyecto Colab
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from kora.selenium import wd
# Importamos sys es el encargado de proveer variables y funcionalidades, directamente relacionadas con el intérprete.
import sys
from datetime import date
# Importamos Time para controlar tiempos de ejecucion
import time
sys.path.insert(0,'/usr/lib/chronium-browser/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)
wd.get('https://www.magitech.pe/laptops.html')
wd.title

'Laptops, Notebooks y Tablets | Magitech.pe'

In [ ]:
#Seleccionamos el sitio web a realizar web scrapping y la obtenemos con el metodo get(), tambien usamos el navegador Chrome para hacer las pruebas de ejecucion
website ='https://www.magitech.pe/laptops.html'
driver = webdriver.Chrome('chromedriver', options=options)
driver.get(website)

In [ ]:
url_to_srap = 'https://www.magitech.pe/laptops.html'
driver.get(url_to_srap)

In [ ]:
html_code = driver.page_source


In [ ]:
# Usamos la libreria  BeautifulSoup y asignamos una lista vacia por cada columna que vamos a necesitar para nuestro dataframe
from bs4 import BeautifulSoup
tecnologias_marca=[]
tecnologias_descripcion=[]
tecnologias_precioNormal=[]
tecnologias_precioInternet=[]

# Con una estructura For hacemos un recorrido de cada etiqueta con la respectiva clase que contiene el texto con la informacion para la columna que deseamos
pg_amount = 12
page =BeautifulSoup(driver.page_source, 'html.parser')
for i in range(0, pg_amount):

  for tecnologias in page.find_all('li', class_='item'):
    descripcion = tecnologias.find('h2', attrs={'class': 'product-name'})
    if descripcion: 
      tecnologias_descripcion.append(descripcion.text)
    else: tecnologias_descripcion.append('')

    precioNormal=tecnologias.find('p', attrs={'class': 'old-price'})
    if precioNormal: 
      tecnologias_precioNormal.append(precioNormal.text)
    else: tecnologias_precioNormal.append('')

    precioInternet=tecnologias.find('p', attrs={'class': 'special-price'})
    if precioInternet: 
      tecnologias_precioInternet.append(precioInternet.text)
    else: tecnologias_precioInternet.append('')

next_btn = driver.find_elements_by_css_selector('.next')
driver.quit()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead


In [ ]:
#Importamos pandas para crear el DataFrame asigando un nombre a cada columna con su respectivo lista creada anteriormente

import pandas as pd
tecnologias_list = pd.DataFrame({
                               'descripcion': tecnologias_descripcion,
                               'precioNormal': tecnologias_precioNormal,
                               'precioInternet': tecnologias_precioInternet
                               
                              })

#tecnologias_list.to_csv('artefactosMagitech.csv', index=None, header=True, encoding='utf-8-sig')

tecnologias_list

,descripcion,precioNormal,precioInternet
0,"Laptop Chuwi MijaBook 13, Intel Celeron N3450 ...","\nPrecio normal:\n\n S/. 2,...","\nOferta\n\n S/. 1,499.00 ..."
1,"Laptop convertible Hyundai HyFlip 2 en 1, Inte...","\nPrecio normal:\n\n S/. 2,...","\nOferta\n\n S/. 1,599.00 ..."
2,"Laptop convertible Hyundai HyFlip 2 en 1, Inte...","\nPrecio normal:\n\n S/. 2,...","\nOferta\n\n S/. 1,799.00 ..."
3,Laptop Convertible Lenovo IdeaPad Flex 5-13IML...,"\nPrecio normal:\n\n S/. 2,...","\nOferta\n\n S/. 1,799.00 ..."
4,Laptop Toshiba Dynabook Tecra A40-G Intel Cele...,"\nPrecio normal:\n\n S/. 2,...","\nOferta\n\n S/. 1,799.00 ..."
...,...,...,...
571,"Laptop Lenovo IdeaPad S340 (14""), Intel Core i...",,
572,Laptop Lenovo IdeaPad 3 (14) Intel Core i5-103...,"\nPrecio normal:\n\n S/. 3,...","\nOferta\n\n S/. 2,999.00 ..."
573,"Laptop Gateway GWTN156 Ultra Slim, Intel Core ...","\nPrecio normal:\n\n S/. 3,...","\nOferta\n\n S/. 2,999.00 ..."
574,"Laptop Gateway GWTN156 Ultra Slim, verde, Inte...","\nPrecio normal:\n\n S/. 3,...","\nOferta\n\n S/. 2,999.00 ..."


#**Limpieza del Dataframe**

**Limpieza de columna precioNormal**

In [ ]:
# Como se vio en el dataframe anterior, los datos aparecen con ciertos caracteres repetitivos extraidos de los saltos de linea dentro de las etiquetas HTML
# Por lo que aqui procedemos a quitar los caracteres del texto que no deseamos en cada columna
tecnologias_list.precioNormal = tecnologias_list.precioNormal.str.replace('\n','')

In [ ]:
tecnologias_list.precioNormal = tecnologias_list.precioNormal.str.replace('Precio','')

In [ ]:
tecnologias_list.precioNormal = tecnologias_list.precioNormal.str.replace('normal','')

In [ ]:
tecnologias_list.precioNormal = tecnologias_list.precioNormal.str.replace(':','')

**Limpieza de columna precioInternet**

In [ ]:
tecnologias_list.precioInternet = tecnologias_list.precioInternet.str.replace('\n','')
tecnologias_list.precioInternet = tecnologias_list.precioInternet.str.replace('Oferta','')


In [ ]:
# Y como resultado tenemos un dataframe con datos mas limpios que el datframe anterior
tecnologias_list

# sin embargo a la columna descripcion se le puede separa en dos: por Marca y Descripcion, para un mejor detalle de producto en el Dataframe Final

,descripcion,precioNormal,precioInternet
0,"Laptop Chuwi MijaBook 13, Intel Celeron N3450 ...","S/. 2,199.00","S/. 1,499.00"
1,"Laptop convertible Hyundai HyFlip 2 en 1, Inte...","S/. 2,199.00","S/. 1,599.00"
2,"Laptop convertible Hyundai HyFlip 2 en 1, Inte...","S/. 2,399.00","S/. 1,799.00"
3,Laptop Convertible Lenovo IdeaPad Flex 5-13IML...,"S/. 2,799.00","S/. 1,799.00"
4,Laptop Toshiba Dynabook Tecra A40-G Intel Cele...,"S/. 2,499.00","S/. 1,799.00"
...,...,...,...
571,"Laptop Lenovo IdeaPad S340 (14""), Intel Core i...",,
572,Laptop Lenovo IdeaPad 3 (14) Intel Core i5-103...,"S/. 3,499.00","S/. 2,999.00"
573,"Laptop Gateway GWTN156 Ultra Slim, Intel Core ...","S/. 3,499.00","S/. 2,999.00"
574,"Laptop Gateway GWTN156 Ultra Slim, verde, Inte...","S/. 3,499.00","S/. 2,999.00"


In [ ]:
# Entonces separamos las cadena de texto de la columna descripcion a partir de la ","
prueba = tecnologias_list["descripcion"].str.split(',', expand=True)
prueba

,0,1,2,3,4,5,6,7
0,Laptop Chuwi MijaBook 13,Intel Celeron N3450 Quad Core 1.10GHz,RAM 8GB,Sólido SSD 256GB,"Pantalla LED 13"" IPS con resolución 3K (3200x...",Windows 10 Home,None,None
1,Laptop convertible Hyundai HyFlip 2 en 1,Intel Celeron N3350 1.10GHz,RAM 4GB,Sólido SSD 64GB PCIe,"LED 11.6"" HD",Windows 10 Pro. Peso 1.1 Kg,None,None
2,Laptop convertible Hyundai HyFlip 2 en 1,Intel Celeron N3350 1.10GHz,RAM 4GB,Sólido SSD 196GB PCIe,"LED 11.6"" HD",Windows 10 Pro. Peso 1.1 Kg,None,None
3,Laptop Convertible Lenovo IdeaPad Flex 5-13IML...,RAM 8GB,Sólido SSD 128GB PCIe,"LED 13.3"" Full HD Táctil",Chrome OS,None,None,None
4,Laptop Toshiba Dynabook Tecra A40-G Intel Cele...,RAM 4GB,Sólido SSD 128GB,"LED 14"" Full HD",Windows 10 Pro,None,None,None
...,...,...,...,...,...,...,...,...
571,"Laptop Lenovo IdeaPad S340 (14"")",Intel Core i5-1035G4 1.1 / 3.7GHz,RAM 8GB,Sólido SSD 256GB PCIe,"Pantalla LED 14"" HD",Windows 10 Home SP,None,None
572,Laptop Lenovo IdeaPad 3 (14) Intel Core i5-103...,RAM 8GB,Sólido SSD 512GB PCIe,"Pantalla LED 14"" Full HD",Windows 10 Home,None,None,None
573,Laptop Gateway GWTN156 Ultra Slim,Intel Core i5-1035G1 1.1GHz,RAM 16GB,Sólido SSD 256GB,"LED 15.6"" Full HD",Windows 10 Home,None,None
574,Laptop Gateway GWTN156 Ultra Slim,verde,Intel Core i5-1035G1 1.1GHz,RAM 16GB,Sólido SSD 256GB,"LED 15.6"" Full HD",Windows 10 Home,None


In [ ]:
#cogemos solo la tabla que necesitamos
Marca = prueba[[0]]
Marca

,0
0,Laptop Chuwi MijaBook 13
1,Laptop convertible Hyundai HyFlip 2 en 1
2,Laptop convertible Hyundai HyFlip 2 en 1
3,Laptop Convertible Lenovo IdeaPad Flex 5-13IML...
4,Laptop Toshiba Dynabook Tecra A40-G Intel Cele...
...,...
571,"Laptop Lenovo IdeaPad S340 (14"")"
572,Laptop Lenovo IdeaPad 3 (14) Intel Core i5-103...
573,Laptop Gateway GWTN156 Ultra Slim
574,Laptop Gateway GWTN156 Ultra Slim


In [ ]:
# Reemplazamos elementos con metodo replace() para la marca
Marca[0] = Marca[0].str.replace('Laptop',"")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# Se usa str.split() para seleccionar partes de una cadena separando por espacios consecutivos
Marca[0] = Marca[0].str.replace('convertible',"")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Marca

,0
0,Chuwi MijaBook 13
1,Hyundai HyFlip 2 en 1
2,Hyundai HyFlip 2 en 1
3,Convertible Lenovo IdeaPad Flex 5-13IML05 Cor...
4,Toshiba Dynabook Tecra A40-G Intel Celeron 52...
...,...
571,"Lenovo IdeaPad S340 (14"")"
572,Lenovo IdeaPad 3 (14) Intel Core i5-1035G1 1....
573,Gateway GWTN156 Ultra Slim
574,Gateway GWTN156 Ultra Slim


In [ ]:

prueba2 = Marca[0].str.split(expand=True)
prueba2

,0,1,2,3,4,5,6,7,8,9,10,11
0,Chuwi,MijaBook,13,None,None,None,None,None,None,None,None,None
1,Hyundai,HyFlip,2,en,1,None,None,None,None,None,None,None
2,Hyundai,HyFlip,2,en,1,None,None,None,None,None,None,None
3,Convertible,Lenovo,IdeaPad,Flex,5-13IML05,Core,i3-10110U,2.1GHz,None,None,None,None
4,Toshiba,Dynabook,Tecra,A40-G,Intel,Celeron,5205U,1.9GHz,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
571,Lenovo,IdeaPad,S340,"(14"")",None,None,None,None,None,None,None,None
572,Lenovo,IdeaPad,3,(14),Intel,Core,i5-1035G1,1.0,/,3.6GHz,None,None
573,Gateway,GWTN156,Ultra,Slim,None,None,None,None,None,None,None,None
574,Gateway,GWTN156,Ultra,Slim,None,None,None,None,None,None,None,None


In [ ]:
Marcaprueba = prueba2[[0]]
Marcaprueba

,0
0,Chuwi
1,Hyundai
2,Hyundai
3,Convertible
4,Toshiba
...,...
571,Lenovo
572,Lenovo
573,Gateway
574,Gateway


In [ ]:
Marcaprueba = Marcaprueba.rename(columns={0:'Marca'})

In [ ]:
Marcaprueba

,Marca
0,Chuwi
1,Hyundai
2,Hyundai
3,Convertible
4,Toshiba
...,...
571,Lenovo
572,Lenovo
573,Gateway
574,Gateway


In [ ]:
# Finalizando con la concatenacion de los elementos procesados para su limpieza
tecnologias_list = pd.concat([Marcaprueba, tecnologias_list], axis=1)

In [ ]:
# Obteniendo el Dataframe como resultado final
tecnologias_list

,Marca,descripcion,precioNormal,precioInternet
0,Chuwi,"Laptop Chuwi MijaBook 13, Intel Celeron N3450 ...","S/. 2,199.00","S/. 1,499.00"
1,Hyundai,"Laptop convertible Hyundai HyFlip 2 en 1, Inte...","S/. 2,199.00","S/. 1,599.00"
2,Hyundai,"Laptop convertible Hyundai HyFlip 2 en 1, Inte...","S/. 2,399.00","S/. 1,799.00"
3,Convertible,Laptop Convertible Lenovo IdeaPad Flex 5-13IML...,"S/. 2,799.00","S/. 1,799.00"
4,Toshiba,Laptop Toshiba Dynabook Tecra A40-G Intel Cele...,"S/. 2,499.00","S/. 1,799.00"
...,...,...,...,...
571,Lenovo,"Laptop Lenovo IdeaPad S340 (14""), Intel Core i...",,
572,Lenovo,Laptop Lenovo IdeaPad 3 (14) Intel Core i5-103...,"S/. 3,499.00","S/. 2,999.00"
573,Gateway,"Laptop Gateway GWTN156 Ultra Slim, Intel Core ...","S/. 3,499.00","S/. 2,999.00"
574,Gateway,"Laptop Gateway GWTN156 Ultra Slim, verde, Inte...","S/. 3,499.00","S/. 2,999.00"


In [ ]:
# Sin embargo se puede eliminar el texto "Laptop" de cada descripcion puesto que es repetitivo y para eso usamos esta linea de codigo
tecnologias_list["descripcion"] = tecnologias_list["descripcion"].str.replace('Laptop '," ")

In [ ]:
tecnologias_list

,Marca,descripcion,precioNormal,precioInternet
0,Chuwi,"Chuwi MijaBook 13, Intel Celeron N3450 Quad C...","S/. 2,199.00","S/. 1,499.00"
1,Hyundai,"convertible Hyundai HyFlip 2 en 1, Intel Cele...","S/. 2,199.00","S/. 1,599.00"
2,Hyundai,"convertible Hyundai HyFlip 2 en 1, Intel Cele...","S/. 2,399.00","S/. 1,799.00"
3,Convertible,Convertible Lenovo IdeaPad Flex 5-13IML05 Cor...,"S/. 2,799.00","S/. 1,799.00"
4,Toshiba,Toshiba Dynabook Tecra A40-G Intel Celeron 52...,"S/. 2,499.00","S/. 1,799.00"
...,...,...,...,...
571,Lenovo,"Lenovo IdeaPad S340 (14""), Intel Core i5-1035...",,
572,Lenovo,Lenovo IdeaPad 3 (14) Intel Core i5-1035G1 1....,"S/. 3,499.00","S/. 2,999.00"
573,Gateway,"Gateway GWTN156 Ultra Slim, Intel Core i5-103...","S/. 3,499.00","S/. 2,999.00"
574,Gateway,"Gateway GWTN156 Ultra Slim, verde, Intel Core...","S/. 3,499.00","S/. 2,999.00"


In [ ]:
# Creamos el dataframe en formato .csv
tecnologias_list.to_csv('artefactosMagitechs.csv', index=None, header=True, encoding='utf-8-sig' )

tecnologias_list

,Marca,descripcion,precioNormal,precioInternet
0,Chuwi,"Chuwi MijaBook 13, Intel Celeron N3450 Quad C...","S/. 2,199.00","S/. 1,499.00"
1,Hyundai,"convertible Hyundai HyFlip 2 en 1, Intel Cele...","S/. 2,199.00","S/. 1,599.00"
2,Hyundai,"convertible Hyundai HyFlip 2 en 1, Intel Cele...","S/. 2,399.00","S/. 1,799.00"
3,Convertible,Convertible Lenovo IdeaPad Flex 5-13IML05 Cor...,"S/. 2,799.00","S/. 1,799.00"
4,Toshiba,Toshiba Dynabook Tecra A40-G Intel Celeron 52...,"S/. 2,499.00","S/. 1,799.00"
...,...,...,...,...
571,Lenovo,"Lenovo IdeaPad S340 (14""), Intel Core i5-1035...",,
572,Lenovo,Lenovo IdeaPad 3 (14) Intel Core i5-1035G1 1....,"S/. 3,499.00","S/. 2,999.00"
573,Gateway,"Gateway GWTN156 Ultra Slim, Intel Core i5-103...","S/. 3,499.00","S/. 2,999.00"
574,Gateway,"Gateway GWTN156 Ultra Slim, verde, Intel Core...","S/. 3,499.00","S/. 2,999.00"


In [ ]:
# Y finalizamos descargando el archivo creado para ser usado como fuente de datos en Power BI
from google.colab import files
files.download('artefactosMagitechs.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>